In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Explore Data

In [57]:

org_df = pd.read_csv("Data/loan_data.csv")
org_df = org_df.sample(frac=1)
org_df.head(3)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
18821,31.0,female,Bachelor,33101.0,7,MORTGAGE,6500.0,EDUCATION,11.11,0.20,7.0,700,Yes,0
43244,59.0,male,Associate,45292.0,38,RENT,5457.0,DEBTCONSOLIDATION,15.84,0.12,20.0,697,No,1
38299,32.0,male,High School,344532.0,8,MORTGAGE,9456.0,VENTURE,12.96,0.03,7.0,567,No,0


In [136]:
loan_df = org_df[['previous_loan_defaults_on_file', 'person_home_ownership', 'loan_status']]
loan_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45000 entries, 18821 to 41938
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   previous_loan_defaults_on_file  45000 non-null  object
 1   person_home_ownership           45000 non-null  object
 2   loan_status                     45000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [154]:
loan_df[loan_df['loan_status'] == 1].shape[0]
loan_df[loan_df['loan_status'] == 0].shape[0]

35000

In [137]:
loan_df.describe().T

,count,mean,std,min,25%,50%,75%,max
loan_status,45000.0,0.222222,0.415744,0.0,0.0,0.0,0.0,1.0


In [138]:
cat_cols_names = loan_df.columns[loan_df.dtypes == 'object'].to_list()
num_cols_names = loan_df.columns[loan_df.dtypes != 'object'].to_list()
print('cat_cols_names= ', cat_cols_names, 'num_cols_names= ', num_cols_names)

cat_cols_names=  ['previous_loan_defaults_on_file', 'person_home_ownership'] num_cols_names=  ['loan_status']


In [139]:
cat_cols = [loan_df.columns.get_loc(c) for c in loan_df.columns[loan_df.dtypes == 'object']]
num_cols = [loan_df.columns.get_loc(c) for c in loan_df.columns[loan_df.dtypes != 'object']]
print('cat_cols_ids= ', cat_cols, 'num_cols_ids= ', num_cols)

cat_cols_ids=  [0, 1] num_cols_ids=  [2]


# Split Data

In [140]:
X = loan_df.drop('loan_status', axis=1).values
y = loan_df[['loan_status']].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=45)

# Preprocess Data

In [123]:
from sklearn.compose import  ColumnTransformer 
from sklearn.pipeline import  Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.naive_bayes import GaussianNB


In [141]:
transform = ColumnTransformer(
 [
  ('loan_fault_encoder', OneHotEncoder(drop='first'), [0]),
  ('home_ownership_encoder', OrdinalEncoder(), [1])
 ]
)

In [142]:
print(X_train[:20])
print(transform.fit_transform(X_train)[:20])

[['No' 'RENT']
 ['Yes' 'RENT']
 ['Yes' 'MORTGAGE']
 ['No' 'RENT']
 ['Yes' 'OWN']
 ['Yes' 'MORTGAGE']
 ['Yes' 'MORTGAGE']
 ['Yes' 'RENT']
 ['Yes' 'MORTGAGE']
 ['Yes' 'RENT']
 ['No' 'RENT']
 ['No' 'MORTGAGE']
 ['Yes' 'MORTGAGE']
 ['Yes' 'MORTGAGE']
 ['Yes' 'MORTGAGE']
 ['Yes' 'RENT']
 ['Yes' 'MORTGAGE']
 ['Yes' 'MORTGAGE']
 ['No' 'RENT']
 ['Yes' 'OWN']]
[[0. 3.]
 [1. 3.]
 [1. 0.]
 [0. 3.]
 [1. 2.]
 [1. 0.]
 [1. 0.]
 [1. 3.]
 [1. 0.]
 [1. 3.]
 [0. 3.]
 [0. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 3.]
 [1. 0.]
 [1. 0.]
 [0. 3.]
 [1. 2.]]


# Tranasform and Process Data

In [143]:
pipe = Pipeline(
 steps= [
  ('preprocessing', transform),
  ('model', GaussianNB())
 ]
)

In [144]:
pipe.fit(X_train, y_train.ravel())

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('loan_fault_encoder',
                                                  OneHotEncoder(drop='first'),
                                                  [0]),
                                                 ('home_ownership_encoder',
                                                  OrdinalEncoder(), [1])])),
                ('model', GaussianNB())])

# Model Evaluation

In [145]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
y_pred = pipe.predict(X_test)

In [150]:
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='micro')
print(accuracy, f1)

0.7314814814814815 0.7314814814814815
